In [29]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import*
from datetime import datetime
import pytz
import cvzone

In [30]:
model=YOLO('yolov8s.pt')

In [31]:
"""output_file = 'output.mp4'
execution_file = 'execution.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_file, fourcc, 30.0, (1020, 500))
execution_out = cv2.VideoWriter(execution_file, fourcc, 30.0, (1020, 500))"""

"output_file = 'output.mp4'\nexecution_file = 'execution.mp4'\nfourcc = cv2.VideoWriter_fourcc(*'mp4v')\nout = cv2.VideoWriter(output_file, fourcc, 30.0, (1020, 500))\nexecution_out = cv2.VideoWriter(execution_file, fourcc, 30.0, (1020, 500))"

In [32]:
my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n") 
#print(class_list)

In [33]:
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)



cap=cv2.VideoCapture('video_output (6).mp4')

# Verifique se o vídeo foi aberto corretamente
if not cap.isOpened():
    print("Erro ao abrir o vídeo.")
    exit()

# Obtenha as informações de shape e FPS do vídeo
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Obtenha o fuso horário do Brasil
timezone = pytz.timezone('America/Sao_Paulo')
# Obtenha a data e hora atual no fuso horário do Brasil
data_hora_atual = datetime.now(timezone)

output_file = 'output.mp4'
execution_file = 'execution.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width,frame_height))
execution_out = cv2.VideoWriter(execution_file, fourcc, fps, (frame_width,frame_height))
print(fps)

3.0


In [34]:
count=0

tracker=Tracker()

cy1=150
cy2=368
offset=120

vh_down = {}
downcount=[]
upcount=[]
vh_up = {}

while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
#    print(results)
    a=results[0].boxes.boxes
    px=pd.DataFrame(a).astype("float")
#    print(px)
    list=[]
             
    for index,row in px.iterrows():
#        print(row)
 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list.append([x1,y1,x2,y2])
    bbox_id=tracker.update(list)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2
#        print(id)
        #se ele toca a L1 e depois L2, ele está descendo
        #L2 e L1, ele está subindo
        #Condição para criar a contagem descendo
        if cy1<(cy+offset) and cy1 > (cy-offset): #vendo se o centroide tocou a L1
            #dicionario para verificar se o carro esta subindo ou descendo
            cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
            vh_down[id]=cy #aqui ele recebe cy, pois estão no mesmo ponto
            #print(cy)
        #fazendo um dicionário para ler a info de cada id dentro da biblioteca
        if id in vh_down:
            print(vh_down)
            if cy2<(cy+offset) and cy2 > (cy-offset): #contaremos somento os veiculos que tocam a segunda linha
                #caso tenha tocado, você add o ponto vermelhor e a ID do objeto
                
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                if downcount.count(id)==0: #aqui estamos garantindo que ele só salva se não tiver a id na lista.
                    downcount.append(id)

        #Condição para criar a contagem subindo 
        if cy2<(cy+offset) and cy2 > (cy-offset): #vendo se o centroide tocou a L2
            cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
            #dicionario para verificar se o carro esta subindo ou descendo
            vh_up[id]=cy #aqui ele recebe cy, pois estão no mesmo ponto
        #fazendo um dicionário para ler a info de cada id dentro da biblioteca
        if id in vh_up:
            if cy1<(cy+offset) and cy1 > (cy-offset): #contaremos somento os veiculos que tocam a segunda linha
                #caso tenha tocado, você add o ponto vermelhor e a ID do objeto
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                if upcount.count(id)==0: #aqui estamos garantindo que ele só salva se não tiver a id na lista.
                    upcount.append(id)
       
    

    cv2.line(frame,(116,cy1),(872,cy1),(255,255,255),1)
    #aqui a linha Y precisar ser igual para que seja uma linha horizontal
    cv2.putText(frame,('L1'),(217,196),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.line(frame,(89,cy2),(927,cy2),(255,255,255),1)
    cv2.putText(frame,('L2'),(190,365),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    d=len(downcount) #fazendo a contagem de carros descendo
    up = len(upcount)
    cv2.putText(frame,str(data_hora_atual),(700,30),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.putText(frame,('Descendo: ')+str(d),(700,60),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.putText(frame,('Subindo: ')+str(up),(700,80),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

    out.write(frame)
    execution_out.write(frame)
    
    cv2.imshow("RGB", frame)
    if cv2.waitKey(0)&0xFF==27:
        break


cap.release()
out.release()
execution_out.release()
cv2.destroyAllWindows()


0: 320x640 1 car, 345.1ms
Speed: 4.1ms preprocess, 345.1ms inference, 1.7ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


{0: 148}



0: 320x640 (no detections), 335.7ms
Speed: 3.2ms preprocess, 335.7ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 320x640 (no detections), 419.3ms
Speed: 3.4ms preprocess, 419.3ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 320x640 1 car, 344.5ms
Speed: 4.5ms preprocess, 344.5ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.



{0: 148, 1: 165}


0: 320x640 1 car, 444.0ms
Speed: 5.5ms preprocess, 444.0ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


...

0: 320x640 3 cars, 1 traffic light, 344.2ms
Speed: 4.4ms preprocess, 344.2ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.



{0: 148, 1: 165, 2: 205, 5: 100, 6: 229, 7: 98, 8: 122, 10: 182, 11: 158, 12: 147, 13: 213, 14: 215}
{0: 148, 1: 165, 2: 205, 5: 100, 6: 229, 7: 98, 8: 122, 10: 182, 11: 158, 12: 147, 13: 213, 14: 215, 15: 138}
{0: 148, 1: 165, 2: 205, 5: 100, 6: 229, 7: 98, 8: 122, 10: 182, 11: 158, 12: 147, 13: 213, 14: 215, 15: 138, 16: 121}


0: 320x640 2 cars, 355.5ms
Speed: 3.6ms preprocess, 355.5ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.



{0: 148, 1: 165, 2: 205, 5: 100, 6: 229, 7: 98, 8: 122, 10: 182, 11: 158, 12: 147, 13: 213, 14: 215, 15: 138, 16: 121, 17: 219}


0: 320x640 1 car, 396.2ms
Speed: 4.3ms preprocess, 396.2ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 320x640 (no detections), 368.8ms
Speed: 2.9ms preprocess, 368.8ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 320x640 (no detections), 360.0ms
Speed: 4.2ms preprocess, 360.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


In [35]:
"""  ### CÓDIGO PARA A PISTA DA DIREITA DESCENDO E DA ESQUERDA SUBINDO


        if 'car' in c:
            list.append([x1,y1,x2,y2])
    bbox_id=tracker.update(list)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2
#        print(id)
        #se ele toca a L1 e depois L2, ele está descendo
        #L2 e L1, ele está subindo
        #Condição para criar a contagem descendo
        if cy1<(cy+offset) and cy1 > (cy-offset): #vendo se o centroide tocou a L1
            #dicionario para verificar se o carro esta subindo ou descendo
            vh_down[id]=cy #aqui ele recebe cy, pois estão no mesmo ponto
        #fazendo um dicionário para ler a info de cada id dentro da biblioteca
        if id in vh_down:
            if cy2<(cy+offset) and cy2 > (cy-offset): #contaremos somento os veiculos que tocam a segunda linha
                #caso tenha tocado, você add o ponto vermelhor e a ID do objeto
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                if downcount.count(id)==0: #aqui estamos garantindo que ele só salva se não tiver a id na lista.
                    downcount.append(id)

        #Condição para criar a contagem subindo 
        if cy2<(cy+offset) and cy2 > (cy-offset): #vendo se o centroide tocou a L2
            #dicionario para verificar se o carro esta subindo ou descendo
            vh_up[id]=cy #aqui ele recebe cy, pois estão no mesmo ponto
        #fazendo um dicionário para ler a info de cada id dentro da biblioteca
        if id in vh_up:
            if cy1<(cy+offset) and cy1 > (cy-offset): #contaremos somento os veiculos que tocam a segunda linha
                #caso tenha tocado, você add o ponto vermelhor e a ID do objeto
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                if upcount.count(id)==0: #aqui estamos garantindo que ele só salva se não tiver a id na lista.
                    upcount.append(id)
       
    

    cv2.line(frame,(116,cy1),(872,cy1),(255,255,255),1)
    #aqui a linha Y precisar ser igual para que seja uma linha horizontal
    cv2.putText(frame,('L1'),(278,319),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.line(frame,(89,cy2),(927,cy2),(255,255,255),1)
    cv2.putText(frame,('L2'),(190,365),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    d=len(downcount) #fazendo a contagem de carros descendo
    up = len(upcount)
    cv2.putText(frame,str(data_hora_atual),(700,30),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.putText(frame,('Descendo: ')+str(d),(700,60),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.putText(frame,('Subindo: ')+str(up),(700,80),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

    out.write(frame)
    execution_out.write(frame)
    
    cv2.imshow("RGB", frame)
    if cv2.waitKey(0)&0xFF==27:
        break
"""

'  ### CÓDIGO PARA A PISTA DA DIREITA DESCENDO E DA ESQUERDA SUBINDO\n\n\n        if \'car\' in c:\n            list.append([x1,y1,x2,y2])\n    bbox_id=tracker.update(list)\n    for bbox in bbox_id:\n        x3,y3,x4,y4,id=bbox\n        cx=int(x3+x4)//2\n        cy=int(y3+y4)//2\n#        print(id)\n        #se ele toca a L1 e depois L2, ele está descendo\n        #L2 e L1, ele está subindo\n        #Condição para criar a contagem descendo\n        if cy1<(cy+offset) and cy1 > (cy-offset): #vendo se o centroide tocou a L1\n            #dicionario para verificar se o carro esta subindo ou descendo\n            vh_down[id]=cy #aqui ele recebe cy, pois estão no mesmo ponto\n        #fazendo um dicionário para ler a info de cada id dentro da biblioteca\n        if id in vh_down:\n            if cy2<(cy+offset) and cy2 > (cy-offset): #contaremos somento os veiculos que tocam a segunda linha\n                #caso tenha tocado, você add o ponto vermelhor e a ID do objeto\n                cv2